# D-WAVE OCEAN

Read the docs for further info: https://docs.ocean.dwavesys.com/en/stable/getting_started.html

**New to Jupyter Notebooks?** JNs are divided into text or code cells. Pressing the **Run** button in the menu bar moves to the next cell. Code cells are marked by an "In: \[\]" to the left; when run, an asterisk displays until code completion: "In: \[\*\]".

## Installing Ocean SDK

D-Wave Ocean SDK (Ocean) is a collection of software that uses D-Wave Machines. It is easy to use various features from Python.

Github: https://github.com/dwavesystems/dwave-ocean-sdk  

Document: https://docs.ocean.dwavesys.com/en/latest/

Guide: https://docs.ocean.dwavesys.com/en/stable/overview/install.html

To install Ocean, use pip as described in the link above, or clone the github repository.

In [ ]:
!pip install dwave-ocean-sdk

In [ ]:
#Advanced users might directly install the latest (development) SDK from the D-Wave GitHub repository using a command such as,
#!pip install git+https://github.com/dwavesystems/dwave-ocean-sdk

In [ ]:
#Alternatively, you can clone the dwave-ocean-sdk repo and install the SDK to your virtual environment; for example:
#!git clone https://github.com/dwavesystems/dwave-ocean-sdk.git
#!cd dwave-ocean-sdk
#!python setup.py install

## Configuring Access to Leap’s Solvers

D-Wave’s **Solver API (SAPI)** provides access to the solvers—compute resources for solving problems, such as a D-Wave quantum computers and quantum-classical hybrid solvers—hosted in the Leap quantum cloud.

You need SAPI credentials to access these resources. These credentials are **a token sting** that identifies your account in Dwave.

### Loading credentials in a runtime variable

We can create a variable that stores the access token string and use it explicitly when calling a solver

In [2]:
#setting the value of your token in a variable, hard-coded in the notebook (not-recommended)
token="my SAPI token"

In [3]:
#importing the access token from a file (recommended)
import credentials as cre
# set the connection information as an option
endpoint = 'https://cloud.dwavesys.com/sapi'
token = cre.token

Example: Use the token to execute a solver

In [7]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler(token=token))
h = {}
J = {('s1', 's2'): -0.5}
sampleset = sampler.sample_ising(h, J, num_reads=1000)
print(sampleset)

  s1 s2 energy num_oc. chain_.
0 -1 -1   -0.5     659     0.0
1 +1 +1   -0.5     341     0.0
['SPIN', 2 rows, 1000 samples, 2 variables]


### Ocean tools env set-up

The dwave-ocean-sdk includes an interactive **CLI program** that steps you through setup a virtual environment with all necesary and interesting tools to work with Dwave systems. (CLI stands for Command Line Interface)

For a full and easy development experience it is recommended that before you start writing code, you complete the setup of your environment.

It is also recommended that you include the two last steps:

Install Contributor Ocean Tools, adds **non-open-source tools** such as the **Problem Inspector**.

In [8]:
#Install optional non-open-source Ocean packages
#see help for available commands and then run the install
!dwave install --help

Usage: dwave install [OPTIONS] [PACKAGES]...

  Install optional non-open-source Ocean packages.

Options:
  -l, --list                   List available contrib (non-OSS) packages
  -a, --all                    Install all contrib (non-OSS) packages
  -u, --update                 Reinstall all installed contrib packages
  -y, --accept-license, --yes  Accept license(s) without prompting
  -v, --verbose                Increase output verbosity (additive, up to 4
                               times)  [default: 0]
  --help                       Show this message and exit.


In [17]:
!dwave install

Nothing to do. Try "dwave install --help".


In the virtual environment you created as part of Installing Ocean Tools, run the **dwave setup** command. Once the environment is configured in the setup, you will not need to include the token or other account params in the code. 

The setup creates a **configuration file** that Ocean primitives will automatically use to include these params for you when using the SAPI.

New users can accept the dwave setup command’s defaults (press Enter) for all prompts except the authentication token, which is displayed on the Leap dashboard for your account. Section Configuring Access to Leap’s Solvers explains how you can update your work environment’s configuration at any time.

In [ ]:
#run in terminal
!dwave setup

Example: Use the config file created in the setup to execute a solver. No token

In [18]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
h = {}
J = {('s1', 's2'): -0.5}
sampleset = sampler.sample_ising(h, J, num_reads=1000)
print(sampleset)

  s1 s2 energy num_oc. chain_.
0 -1 -1   -0.5     363     0.0
1 +1 +1   -0.5     637     0.0
['SPIN', 2 rows, 1000 samples, 2 variables]


#### Working with the config file

To manage or change the environment configuration file at any time we use the `dwave config command` also available in the DWave CLI program

In [19]:
#see all command available
!dwave config --help

Usage: dwave config [OPTIONS] COMMAND [ARGS]...

  Create, update or inspect cloud client configuration file(s).

Options:
  --help  Show this message and exit.

Commands:
  create   Create or update cloud client configuration file.
  inspect  Inspect existing configuration/profile.
  ls       List configuration files detected (and/or examined paths).


In [20]:
#inspect the config file
!dwave config inspect

Configuration file: auto-detected
Profile: auto-detected
---
token = DEV-3c7fa50472bd2e266912fba5cffb8415dbd8e6e2


In [21]:
#see where it is
!dwave config ls

/home/codespace/.config/dwave/dwave.conf


In [22]:
#use linux cat command to see it
!cat /home/codespace/.config/dwave/dwave.conf

[defaults]
token = DEV-3c7fa50472bd2e266912fba5cffb8415dbd8e6e2



In [ ]:
#execute in terminal
#IMPORTANT: do not include [] or "" in any param
!dwave config create --full

In [25]:
!cat /home/codespace/.config/dwave/dwave.conf

[defaults]
token = DEV-3c7fa50472bd2e266912fba5cffb8415dbd8e6e2

[europe]
region = eu-central-1



Example: using dwave CLI to see the available solvers, their parameters, and properties.

In [26]:
#na-west-1 eu-central-1
!dwave solvers  --list --all --region na-west-1

hybrid_binary_quadratic_model_version2
hybrid_discrete_quadratic_model_version1
hybrid_constrained_quadratic_model_version1
Advantage2_prototype1.1
Advantage_system4.1
Advantage_system6.3


Example: using dwave-cloud-client to query for hybrid solvers.

IMPORTANT: having different config profiles you can select different config params specifying the profile explicitly

In [29]:
from dwave.cloud import Client
#specify the profile you are using in the Client command -> check that in europe there are not hybrid solvers
with Client.from_config(profile="defaults") as client:
   solvers_list=client.get_solvers(hybrid=True)

for s in solvers_list:
  print(s)

BQMSolver(id='hybrid_binary_quadratic_model_version2')
DQMSolver(id='hybrid_discrete_quadratic_model_version1')
CQMSolver(id='hybrid_constrained_quadratic_model_version1')


Example: using dwave-cloud-client to query supported regions.

In [28]:
with Client.from_config(profile="defaults") as client:
   regions = client.get_regions()
   for code, info in regions.items():
       print(f"{info['name']} ({code}): {info['endpoint']}")

North America (na-west-1): https://na-west-1.cloud.dwavesys.com/sapi/v2/
Europe (eu-central-1): https://eu-central-1.cloud.dwavesys.com/sapi/v2/


/tmp/ipykernel_34307/1938834912.py:2: DeprecationWarning: `Client.get_regions` method is deprecated since dwave-cloud-client 0.11.0, and will be removed in 0.13.0. Use `dwave.cloud.regions.get_regions` for greater flexibility instead.
  regions = client.get_regions()


#### Verify configuration of the virtual environment

To Verify your configuration in the virtual environment, you can test that your solver access is configured correctly with the interactive CLI.

In your virtual environment, run the dwave ping command

In [30]:
!dwave ping --client qpu

Using endpoint: https://eu-central-1.cloud.dwavesys.com/sapi/v2/
Using region: eu-central-1
Using solver: Advantage_system5.4
Submitted problem ID: 558b59a3-a5de-4288-afe0-8f1b5937be09

Wall clock time:
 * Solver definition fetch: 544.254 ms
 * Problem submit and results fetch: 513.393 ms
 * Total: 1057.647 ms

QPU timing:
 * post_processing_overhead_time = 5320.0 us
 * qpu_access_overhead_time = 1600.01 us
 * qpu_access_time = 14954.99 us
 * qpu_anneal_time_per_sample = 20.0 us
 * qpu_delay_time_per_sample = 20.54 us
 * qpu_programming_time = 14896.85 us
 * qpu_readout_time_per_sample = 17.6 us
 * qpu_sampling_time = 58.14 us
 * total_post_processing_time = 5320.0 us


or run a random toy problem to see if everything works

In [41]:
!dwave sample --random-problem

Using endpoint: https://eu-central-1.cloud.dwavesys.com/sapi/v2/
Using region: eu-central-1
Using solver: Advantage_system5.4
Using biases: [(30, -3.8475967349032167), (31, 1.8854204005402808), (32, 1.4088168376768087), (33, 0.0277039416759814...
Using couplings: [((1333, 5391), -0.36685743768212364), ((1469, 5437), 0.6668017964640645), ((2336, 5022), 0.30983573...
Sampling parameters: {'label': 'dwave sample'}
Submitted problem ID: ec498aa0-d129-4fb0-a343-5c7a9c3f1201
Samples: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, -1...
Occurrences: [1]
Energies: [-14138.182594183952]

Wall clock time:
 * Solver definition fetch: 674.155 ms
 * Problem submit and results fetch: 537.120 ms
 * Total: 1211.274 ms

QPU timing:
 * post_processing_overhead_time = 283.0 us
 * qpu_access_overhead_time = 1836.51 us
 * qpu_access_time = 15049.49 us
 * qpu_anneal_time_per_sample = 20.0 us
 * qpu_delay_time_per_sample = 20.54 us
 * qpu_programming